In [1]:
!wget https://vision.cs.hacettepe.edu.tr/files/recipeqa/train.json
!wget https://vision.cs.hacettepe.edu.tr/files/recipeqa/val.json
!wget https://vision.cs.hacettepe.edu.tr/files/recipeqa/test.json

--2021-01-07 16:17:59--  https://vision.cs.hacettepe.edu.tr/files/recipeqa/train.json
Resolving vision.cs.hacettepe.edu.tr (vision.cs.hacettepe.edu.tr)... 193.140.236.29
Connecting to vision.cs.hacettepe.edu.tr (vision.cs.hacettepe.edu.tr)|193.140.236.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 158656297 (151M) [application/json]
Saving to: ‘train.json.1’

train.json.1        100%[===================>] 151.31M  20.3MB/s    in 9.1s    

2021-01-07 16:18:08 (16.7 MB/s) - ‘train.json.1’ saved [158656297/158656297]

--2021-01-07 16:18:08--  https://vision.cs.hacettepe.edu.tr/files/recipeqa/val.json
Resolving vision.cs.hacettepe.edu.tr (vision.cs.hacettepe.edu.tr)... 193.140.236.29
Connecting to vision.cs.hacettepe.edu.tr (vision.cs.hacettepe.edu.tr)|193.140.236.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19501603 (19M) [application/json]
Saving to: ‘val.json.1’

val.json.1          100%[===================>]  18.60M  9.85MB/

In [2]:
import json
import pandas as pd

In [3]:
def process_file(file:str)->pd.DataFrame:
    with open(file,'r') as f:
        file_json = json.load(f)
        file_data = [{'body':d['body'],
            'title':d['title']
            } for b in file_json['data'] for d in b['context'] if d.get('title')]
        file_df = pd.DataFrame(file_data)
        return file_df

In [4]:
train_df = process_file('train.json')
val_df = process_file('val.json')
test_df = process_file('test.json')

In [5]:
train_df['title'].value_counts()

Ingredients                                    5641
Enjoy!                                         1558
Bake                                            914
Serve                                           614
Enjoy                                           557
                                               ... 
Whisk White                                       1
Marinate the Meet                                 1
Preheat the Oven and Prep the Sweet Potato!       1
Adding the Acid                                   1
Refill Feed/Empty the Bucket Periodically         1
Name: title, Length: 31473, dtype: int64

#### Plans? We Don't Need No Stinkin' Plans!

In [6]:
train_df[train_df['title']=="Plans? We Don't Need No Stinkin' Plans!"]['body'].values[0]

"But I would strongly suggest you at least have some sort of idea of how you want to decorate this cake. I didn't and just made it up as I went. Doodle out a few ideas of how you want it to look (if you want. It's not required, and I'm not your mother. Have fun with it!) Once you have an idea in mind, begin kneading and softening your fondant to roll it out. If you want grey fondant, use a LOT of white, and a pinch of black and begin kneading and mixing. If you have never worked with fondant before, you can find any number of tutorials on YouTube or cake websites or blogs. It isn't hard, but it does take a lot of work (and forearm strength) to mix colors and get it all rolled out. You can also use your fondant colors to color the fondant the way you want. I stuck with a flat grey all over, and then went back and painted each piece, to save on color-mixing effort. Once you have your fondant rolled out, remove the cake from the freezer (be sure to remove the toothpicks or skewers if you 

# Deleting Duplicates

In [7]:
train_df = train_df.drop_duplicates(['body'])
val_df = val_df.drop_duplicates(['body'])
test_df = test_df.drop_duplicates(['body'])

In [8]:
# train_df = train_df.append(val_df)

In [9]:
train_df.shape[0], val_df.shape[0], test_df.shape[0]

(47704, 5795, 5863)

In [10]:
train_df.to_csv('train.csv')
val_df.to_csv('val.csv')
test_df.to_csv('test.csv')

In [11]:
train_df.head()

,body,title
0,3 until 5 whole vanilla beans250 gram of veget...,Ingredients Halal Vanilla Extract
1,Scrape Vanilla Beans and get the seeds into ve...,Scrape Vanilla Beans
2,Vanilla Beans Seed and Vegetable Glycerin,Vegetable Glycerin and Vanilla Beans
3,Whole Vanilla Beans put in a bottle with seeds...,Vanilla Beans Can Use With Vegetable Glycerin
4,Ingredients:-250 g dark chocolate;-2 dl wippin...,WHAT YOU NEED


##Modelling

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchtext.data import Field, BucketIterator
from torchtext import data

import spacy
import numpy as np

import random
import math
import time

In [13]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [14]:
spacy_en = spacy.load('en')

In [15]:
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [16]:
SRC = Field(tokenize= tokenize_en, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

TRG = Field(tokenize = tokenize_en, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [18]:
fields = [('Unnamed: 0', None), ('body', SRC), ('title', TRG)]

In [19]:
train_data, valid_data, test_data = data.TabularDataset.splits(format='csv',
                           path = './',
                           skip_header=True,
                           train='train.csv',
                           validation='val.csv',
                           test='test.csv',
                           fields=fields)

In [20]:
len(train_data), len(valid_data), len(test_data)

(47704, 5795, 5863)

In [21]:
print(vars(train_data.examples[100]))

{'body': ['in', 'another', 'small', 'bowl', ',', 'combine', 'the', 'soy', 'sauce', ',', 'sake', ',', 'rice', 'wine', 'vinegar', ',', 'brown', 'sugar', ',', 'msg', ',', 'and', '1/4', 'tsp', 'salt', 'for', 'the', 'noodle', 'sauce', 'part', '2', '.'], 'title': ['prepare', 'noodle', 'sauce', 'part', '2']}


In [22]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [23]:
BATCH_SIZE = 8
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort = False,
    # sort_key = lambda x:len(x.body), 
    device = device)

In [24]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded)
                
        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden

In [25]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        #attention= [batch size, src len]
        
        return F.softmax(attention, dim=1)

In [26]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0)

In [27]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [28]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

In [29]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(24920, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(4274, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=4274, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [30]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 21,570,226 trainable parameters


In [31]:
optimizer = optim.Adam(model.parameters())

In [32]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [33]:
from tqdm.auto import tqdm

def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in tqdm(enumerate(iterator)):
        
        src = batch.body
        trg = batch.title
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [34]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.body
            trg = batch.title

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [35]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [37]:
N_EPOCHS = 5
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')


Epoch: 01 | Time: 8m 50s
	Train Loss: 4.395 | Train PPL:  81.023
	 Val. Loss: 4.122 |  Val. PPL:  61.709



Epoch: 02 | Time: 8m 55s
	Train Loss: 3.702 | Train PPL:  40.508
	 Val. Loss: 3.966 |  Val. PPL:  52.782



Epoch: 03 | Time: 8m 56s
	Train Loss: 3.276 | Train PPL:  26.462
	 Val. Loss: 3.965 |  Val. PPL:  52.742



Epoch: 04 | Time: 8m 56s
	Train Loss: 2.892 | Train PPL:  18.027
	 Val. Loss: 4.153 |  Val. PPL:  63.636



Epoch: 05 | Time: 8m 56s
	Train Loss: 2.571 | Train PPL:  13.084
	 Val. Loss: 4.359 |  Val. PPL:  78.157


In [38]:
model.load_state_dict(torch.load('tut3-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.962 | Test PPL:  52.545 |
